# Wat is Owenership in Rust?

- Ownership is a unique feature of Rust that manages memory safety without a garbage collector
- It allows developers to write efficient and safe code by enforcing rules on how memory is allocated and deallocated
- In Rust, every value has a single owner, and when the owner goes out of scope, the value is automatically dropped, freeing the memory
- This system helps prevent common programming errors such as null pointer dereferencing and memory leaks
- Rust compiler checks ownership rules at compile time, ensuring that code adheres to these principles and is memory safe
    - if any ownership rules are violated, the code will not compile, providing a strong guarantee of memory safety
- ownership is a new concept for many programmers, it does take some time to get used to
- to understand data ownership, one must understand the stack and the heap segments of memory layout

## The Stack and the Heap

- Rust programs have 5 main memory regions: TEXT, DATA, BSS, HEAP, and STACK
    - the same memory layout used by C and C++ programs
- Each region serves a specific purpose in how memory is allocated and managed during program execution

```text
Low addresses
┌──────────────────────────────┐
│            TEXT              │
│  (program instructions)      │
│  fn main(), functions, code  │
├──────────────────────────────┤
│            DATA              │
│  initialized globals         │
│  static X: i32 = 5;          │
├──────────────────────────────┤
│             BSS              │
│  zero-initialized globals    │
│  static mut COUNT: i32 = 0;  │
├──────────────────────────────┤
│             HEAP             │  ↓ grows towards STACK
│  Vec, String, Box allocations│
│  dynamic memory              │
│                              │
│            (free)            │
│                              │
├──────────────────────────────┤
│             STACK            │  ↑ grows towards HEAP
│  function frames             │
│  local variables             │
│  return addresses            │
└──────────────────────────────┘
High addresses
```

### TEXT Segment
- The TEXT segment contains the compiled program's machine code, including the main function and any other functions defined in the program
- This segment is read-only and is loaded into memory when the program starts

### DATA Segment
- The DATA segment holds initialized global and static variables, such as `static X: i32 = 5;`
- These variables have a fixed value that is set at compile time and remain in memory for the duration of the program

### BSS Segment
- The BSS segment contains uninitialized global and static variables, such as `static mut COUNT: i32 = 0;`
- These variables are initialized to zero at runtime and also remain in memory for the duration of the program

### HEAP Segment
- The HEAP segment is used for dynamic memory allocation, such as when using `Vec`, `String`, or `Box`
- Memory in the HEAP is allocated and deallocated at runtime, and it grows towards the higher address or STACK region
- The HEAP is managed by Rust's ownership system
- When a value is allocated on the HEAP, it is owned by a variable, and when that variable goes out of scope, the memory is automatically freed
- This allows Rust to manage memory safely without a garbage collector, preventing memory leaks and dangling pointers
- heap is less organized and slower to access than the STACK, so it's generally more efficient to use the STACK for small, short-lived data and the HEAP for larger, long-lived data
- the HEAP can grow and shrink dynamically, but it can also lead to fragmentation if not managed properly, which can reduce performance over time
    - think of the HEAP as a pile of plates, where you can add or remove plates from anywhere in the pile

### STACK Segment
- The STACK segment is used for function call frames, local variables, and return addresses
- Each time a function is called, a new frame is pushed onto the STACK, and when the function returns, the frame is popped off
- The STACK grows towards the lower address or HEAP region
- The STACK is organized and fast to access, making it ideal for small, short-lived data
- However, the STACK has a limited size, and if too much data is allocated on the STACK, it can lead to a stack overflow
- local variables and function parameters are typically pushed on the STACK in LIFO (Last In, First Out) order, meaning the most recently called function's variables are at the top of the STACK
- when a function returns, its frame is popped off the STACK, and the memory used by its local variables is freed automatically
- think STACK as a stack of plates, where you can only add or remove the top plate

## Ownership Rules

- Each value in Rust has an `owner` - the variable that owns it
- There can only be one owner at a time
- When the owner goes out of scope, the value is dropped and the memory is freed
- These rules are enforced at compile time, ensuring memory safety without a garbage collector
- If any ownership rules are violated, the code will not compile, providing a strong guarantee of memory safety


## Variable Scope

- a scope is a region of the program where a variable is valid and can be accessed
- in Rust, variables are valid from the point they are declared until the end of the block in which they are declared
- when a variable goes out of scope, it is dropped and the memory it occupies is freed
- there are 3 levels of scope in Rust: global scope, function scope, and block scope
    - global scope: variables declared outside of any function or block, accessible throughout the entire program
    - function scope: variables declared within a function, accessible only within that function
    - block scope: variables declared within a block (enclosed by curly braces `{}`), accessible only within that block

In [ ]:
fn product(x: i32, y: i32) -> i32 {
    // x and y are parameters with local scope to the function
    let result: i32 = x * y; // result is a local variable
    return result; // returning the result to the caller
}

In [ ]:
{   // ans not valid here
    let ans = product(5, 10); 
    // ans is valid this point forward until the end of this scope
    println!("The product of 5 and 10 is: {}", ans);
} // end of the scope for ans, memory used by ans is freed here
// ans is no longer valid here, trying to use ans here would result in a compile-time error

The product of 5 and 10 is: 50


()

## The String Type

- here we'll use the String type to demonstrate the ownership model
- the model applies to all advanced data types that use dymanic memory in heap to store data
- non-ownership and other aspects of Strings are covered in the Strings chapter

### Immutable String

- literal hard-coded string that is compiled as part of the final executable
- immutable string literals are fast and efficient
- immutable strings are not suitable for every situation in which we may want to use text

In [ ]:
// literal named string
let s = "Hello!";

In [5]:
// can't mutate s; compile-time error
s.push_str(", world!");

Error: no method named `push_str` found for reference `&str` in the current scope

## Mutable String

- Rust provides `String` type to store text that is not known during compile-time
- user input text data can be strored in the `String` objects
- to support a mutable, growable piece of text, we need to allocate an amount of data on the heap, unknown at compile time to hold the contents
- memory must be requested from the memory allocator at runtime
    - programmers explictly do this in Rust
- we need a way of returning this memory back to the allocator when we're done with our String
    - No Garbage Collector, but the memory is automatically returned once the variable that owns goes out of scope
    - Rust calls the function `drop` automatically at the end of the scope

- `::` is a name resolution operator

In [ ]:
{ 
    let mut s1 = String::from("Hello"); // requests memory from heap
    s1.push_str(" World!"); // appends a literal to a String
    println!("{s}");
}
// s1 is deallocated/returned back to the allocator

Hello!


()

## Variables and Data Moving and Copying

- multiple variables can interact with the same data in different ways
- what happens when one variable is copied to another?
- when working with known fixed sized simple types such as integers, chars, and floats, and string literals data is copied and pushed on the stack
- deep copy is performed on the primitive simple types

In [8]:
let x = 5;
let y = x;
println!("x = {x}, y = {y}");

x = 5, y = 5


In [ ]:
use std::ptr;
// & - gives access to the address of the variable
{
    println!("{}", ptr::eq(&x, &y)); // false;
}

false


()

In [10]:
let a = "apple";
let b = a;
println!("a = {a}, b = {b}");

a = apple, b = apple


### Copying the mutable `String`

- when a mutable `String` is declared, it creates an object with 3 parts

```rust
let s1 = String::from("hello");
let s2 = s1;
```
<img src="./assets/string_1.svg" width="400"/>

- when `s1` is copied to `s2`, you'd think that the value "hello" is copied and assigned to `s2`, but this doesn't quite happen!
- only the ptr, len and capacity that are on the stack are copied
- this is called shallow copy! meaning, it doesn't copy the data on the heap that the pointer refers to

<img src="./assets/string_2.svg" width="400"/>

- copying the data in the heap could be expensive if the data is large!
- in Rust, `s2` takes over the ownership of data of `s1` making `s1` no longer valid
    - this avoid double free if both `s1` and `s2` go out of scope and try to free the same memory twice

In [ ]:
let s1 = String::from("hello");
let s2 = s1; // ownership of s1 is transferred to s2

In [12]:
// this should be an error!
println!("{s1}, world!");

Error: cannot find value `s1` in this scope

In [23]:
// deep copy
use std::ptr;
{
    let s3 = s2.clone();
    println!("{s2}, world!");
    println!("{}", ptr::eq(s2.as_ptr(), s3.as_ptr())); // false (not alias)
}

hello, world!
false


()

In [ ]:
// shallow copy without transferring owernship
{
    let s4 = &s3;
    println!("{}", ptr::eq(s4.as_ptr(), s3.as_ptr())); // true (alias)
}

true


()

## Rule of Thumb

### Need independent ownership? → clone()
### Just need to read? → borrow (&String or &str)
### Want to transfer ownership? → move

## Functions and Ownership
- the mechanics of passing a value to a function are similar to assigning a value to a variable
- if the type implements the `Copy` trait, the value is copied into the function
- if the type does not implement the `Copy` trait, the value is moved into the function
- when a value is moved into a function, the original variable is no longer valid and cannot be used after the function call
- when a value is copied into a function, the original variable remains valid and can be used after the function call
- when a value is moved into a function, the function takes ownership of the value, and when the function ends, the value is dropped and the memory is freed.

- Here are some of the types that implement Copy:
    - All the integer types, such as u32.
    - The Boolean type, bool, with values true and false.
    - All the floating-point types, such as f64.
    - The character type, char.
    - Tuples, if they only contain types that also implement Copy. For example, (i32, i32) implements Copy, but (i32, String) does not.

In [6]:
fn main() {
    let s = String::from("hello");  // s comes into scope

    takes_ownership(s);             // s's value moves into the function...
                                    // ... and so is no longer valid here

    let x = 5;                      // x comes into scope

    makes_copy(x);                  // Because i32 implements the Copy trait,
    
                                    // x does NOT move into the function,
    println!("{x}");                // so it's okay to use x afterward.

} // Here, x goes out of scope, then s. However, because s's value was moved,
  // nothing special happens.

fn takes_ownership(some_string: String) { // some_string comes into scope
    println!("some_string = {some_string}");
} // Here, some_string goes out of scope and `drop` is called. The backing
  // memory is freed.

fn makes_copy(some_integer: i32) { // some_integer comes into scope
    println!("some_integer = {some_integer}");
} // Here, some_integer goes out of scope. Nothing special happens.

main();

some_string = hello
some_integer = 5
5


## Return Values and Scope
- when a function returns a value, the ownership of that value is transferred to the caller 
- if the returned value is of a type that implements the `Copy` trait, it is copied into the caller, and the original value in the function is dropped when the function ends
- if the returned value is of a type that does not implement the `Copy` trait, it is moved into the caller, and the original value in the function is no longer valid after the function returns


In [9]:
fn main() {
    let s1 = gives_ownership();        // gives_ownership moves its return
                                       // value into s1

    let s2 = String::from("hello");    // s2 comes into scope

    let s3 = takes_and_gives_back(s2); // s2 is moved into
                                       // takes_and_gives_back, which also
    println!("s3 = {s3}");               // moves its return value into s3
} // Here, s3 goes out of scope and is dropped. s2 was moved, so nothing
  // happens. s1 goes out of scope and is dropped.

fn gives_ownership() -> String {       // gives_ownership will move its
                                       // return value into the function
                                       // that calls it

    let some_string = String::from("yours"); // some_string comes into scope

    some_string                        // some_string is returned and
                                       // moves out to the calling
                                       // function
}

// This function takes a String and returns a String.
fn takes_and_gives_back(a_string: String) -> String {
    // a_string comes into scope
    a_string  // a_string is returned and moves out to the calling function
}

main();

s3 = hello


## References and Borrowing
- Rust allows you to have references to values without taking ownership of them, which is called borrowing
- a reference is created using the `&` symbol, and it allows you to read without taking ownership and are immutable by default
- when you borrow a value, you can have either one mutable reference or any number of immutable references, but not both at the same time
- this borrowing system allows Rust to ensure memory safety while still allowing for flexible and efficient code
- a reference is like a const pointer in C/C++, but it is guaranteed to always be valid and not dangling, which prevents many common programming errors

In [11]:
fn main() {
    let s1 = String::from("hello");
    let len = calculate_length(&s1);

    println!("The length of '{s1}' is {len}.");
}

fn calculate_length(s: &String) -> usize {
    s.len()
}

main();

The length of 'hello' is 5.


## Mutable References

- to modify a value through a reference, you need to create a mutable reference using `&mut`
- you can only have one mutable reference to a value in a particular scope, which prevents data races 

In [13]:
fn main() {
    let mut s = String::from("hello");

    change(&mut s);
    println!("s = {s}");
}

fn change(some_string: &mut String) {
    some_string.push_str(", world");
}

main();

s = hello, world


In [15]:
// the following will fail
{
    let mut s = String::from("hello");

    let s1 = &mut s;
    let s2 = &mut s;

    println!("{s1}, {s2}");
}

Error: cannot borrow `s` as mutable more than once at a time

In [18]:
// the following is fine
{
    let mut s = String::from("hello");

    {
        let r1 = &mut s;
    } // r1 goes out of scope here, so we can make a new reference with no problems.

    let r2 = &mut s;
    println!("r2 = {r2}");
}

r2 = hello


()